In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 24.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from sentence_transformers import SentenceTransformer

In [83]:
import pandas as pd

playlists = pd.read_csv('playlists.csv')
titles = playlists['title'].values.tolist()

In [27]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(titles)
print(embeddings)

[[ 0.01121666  0.00339286  0.00341788 ... -0.00339     0.06875719
   0.11063983]
 [-0.08927014 -0.09901916 -0.02933889 ...  0.074058   -0.03957585
   0.07006022]
 [-0.04976848  0.05802613  0.0233979  ...  0.03695691 -0.03862747
   0.03469087]
 ...
 [-0.07699248 -0.05384574 -0.03212197 ...  0.03566454 -0.01031594
   0.04483033]
 [ 0.03313463  0.11171297  0.03359863 ... -0.01501594  0.00949897
   0.02914794]
 [-0.03422984 -0.02196825  0.01068655 ... -0.01350442  0.09874579
  -0.05781006]]


In [29]:
# get pca first 10 components of embeddings
from sklearn.decomposition import PCA
import numpy as np

pca = PCA()

# Fit the data
X_pca = pca.fit_transform(embeddings)

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Print cumulative explained variance
for i, cumulative_ratio in enumerate(cumulative_explained_variance):
    print(f"Cumulative Variance up to Component {i + 1}: {cumulative_ratio:.4f}")

Cumulative Variance up to Component 1: 0.0523
Cumulative Variance up to Component 2: 0.0943
Cumulative Variance up to Component 3: 0.1268
Cumulative Variance up to Component 4: 0.1543
Cumulative Variance up to Component 5: 0.1776
Cumulative Variance up to Component 6: 0.1987
Cumulative Variance up to Component 7: 0.2193
Cumulative Variance up to Component 8: 0.2380
Cumulative Variance up to Component 9: 0.2550
Cumulative Variance up to Component 10: 0.2710
Cumulative Variance up to Component 11: 0.2868
Cumulative Variance up to Component 12: 0.3015
Cumulative Variance up to Component 13: 0.3157
Cumulative Variance up to Component 14: 0.3287
Cumulative Variance up to Component 15: 0.3413
Cumulative Variance up to Component 16: 0.3532
Cumulative Variance up to Component 17: 0.3645
Cumulative Variance up to Component 18: 0.3753
Cumulative Variance up to Component 19: 0.3855
Cumulative Variance up to Component 20: 0.3957
Cumulative Variance up to Component 21: 0.4053
Cumulative Variance up

In [50]:
embeddings_df = pd.DataFrame(embeddings)
playlists = pd.read_csv('playlists.csv')
playlists = pd.concat([playlists, embeddings_df], axis=1)
playlists.to_csv('playlists_embeddings.csv', index=False)

In [79]:
embeddings_df = pd.DataFrame(embeddings)
playlists = pd.read_csv('playlists.csv')
playlists = pd.concat([playlists.get(['index', 'title']), embeddings_df], axis=1)
playlists.to_csv('playlists_embeddings_only.csv', index=False)

In [ ]:
playlists_df = pd.read_csv('playlists_embeddings_only.csv')

# Train test split
np.random.seed(42)
msk = np.random.rand(len(playlists_df)) < 0.8
playlists_df, test_playlists_df = playlists_df[msk], playlists_df[~msk]

headers = playlists_df.columns.values.tolist()

print(headers)

['index', 'title', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131'

/var/folders/9f/k7cll8r177l8wx_yj2fhmld40000gn/T/ipykernel_39357/3089140968.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  playlists_df = pd.read_csv('playlists_embeddings_only.csv')


In [66]:
# euclidean distance from "christmas"
from scipy.spatial import distance

christmas = model.encode(['christmas'])
distances = []
for i in range(len(X_pca)):
    distances.append(distance.euclidean(embeddings[i], christmas[0]))
print(sum([1 for i in distances if i < 0.001]))
print(sum([1 for i in distances if i < 0.1]))
print(sum([1 for i in distances if i < 0.5]))
print(sum([1 for i in distances if i < 1]))

214
214
222
662


In [82]:
christmas = pca.transform(model.encode(['christmas']))
distances = []
for i in range(len(X_pca)):
    distances.append(distance.euclidean(X_pca[i][:15], christmas[0][:15]))
print(sum([1 for i in distances if i < 0.001]))
print(sum([1 for i in distances if i < 0.1]))
print(sum([1 for i in distances if i < 0.5]))
print(sum([1 for i in distances if i < 1]))

214
214
385
25373


In [85]:
embeddings_small_df = pd.DataFrame(X_pca[:, :15])

playlists = pd.read_csv('playlists.csv')
playlists = pd.concat([playlists, embeddings_small_df], axis=1)
playlists.to_csv('playlists_embeddings_small.csv', index=False)